In [1]:
import os

In [2]:
%pwd

'd:\\Bhavadeesh\\Education\\MLOps-Udemy\\projects\\data_science_project\\research'

In [3]:
os.chdir('../')
%pwd

'd:\\Bhavadeesh\\Education\\MLOps-Udemy\\projects\\data_science_project'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [ ]:
from src.data_science.constants import *
from src.data_science.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self,
        config_file_path: Path = CONFIG_FILE_PATH,
        params_file_path: Path = PARAMS_FILE_PATH,
        schema_file_path: Path = SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )
        return data_ingestion_config

In [6]:
# Component - Data Ingestion
import urllib.request as requests
from src.data_science import logger
import zipfile


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    # Downloading the zip file from the source URL
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = requests.urlretrieve(
                self.config.source_URL, self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: {headers}")
        else:
            logger.info(f"File already exists at {self.config.local_data_file}")

    def extract_zip_file(self):
        """
        zip_file_path = str
        Extract the zip file into the data directory
        Function returns None
        """
        unzip_dir = self.config.unzip_dir
        os.makedirs(unzip_dir, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_dir)
            logger.info(f"Extracted all the files in {unzip_dir}")

    def __repr__(self) -> str:
        return f"DataIngestion({self.config})"

In [7]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-04-27 12:02:38,898: INFO: common: YAML file: config\config.yaml loaded successfully]
[2025-04-27 12:02:38,900: INFO: common: YAML file: params.yaml loaded successfully]
[2025-04-27 12:02:38,905: INFO: common: YAML file: schema.yaml loaded successfully]
[2025-04-27 12:02:38,907: INFO: common: Directory created at: artifacts]
[2025-04-27 12:02:38,908: INFO: common: Directory created at: artifacts/data_ingestion]
[2025-04-27 12:02:38,910: INFO: 3729056568: File already exists at artifacts\data_ingestion\data.zip]
[2025-04-27 12:02:38,913: INFO: 3729056568: Extracted all the files in artifacts\data_ingestion]
